### Import Libraries

In [1]:
# import sys
# warp_egg_path = '/opt/anaconda3/lib/python3.7/site-packages/warpctc_pytorch-0.1-py3.7-linux-x86_64.egg/warpctc_pytorch/__init__.py'
# # warp_egg_path = '/usr/local/lib/python3.6/dist-packages/warpctc_pytorch-0.1-py3.6-linux-x86_64.egg'
# sys.path.append(warp_egg_path)
# from warpctc_pytorch import CTCLoss

In [1]:
# import self-defined jupyter notebook 
import nbimporter
import utils
import model

In [2]:
# from __future__ import print_function
import random
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch
import string
import numpy as np
import torch.backends.cudnn as cudnn
import torch.optim as optim
import random
from tqdm import tqdm
import torchvision.models as models
from torch.autograd import Variable
from sklearn.metrics import confusion_matrix

import pandas as pd
import editdistance
from tqdm import trange
import matplotlib.pyplot as plt

### Set Hyperparameters

In [6]:
#hyperparameters
BATCH_SIZE = 64
EPOCH = 30
IMGH = 32
IMGW = 100

### Configure Dataloader

In [4]:
train_transformer = transforms.Compose([
    transforms.Grayscale(),  
    transforms.Resize((IMGH,IMGW)),
    transforms.ToTensor()])  # transform it into a torch tensor

In [5]:
# len(os.listdir('data'))

In [4]:
PATH_TRAIN = "data"

In [7]:
# train-validation split (80% train, 20% validation)
from sklearn.model_selection import train_test_split
n = range(len(os.listdir(PATH_TRAIN)))
train_idx, val_idx = train_test_split(n, train_size=0.8)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [8]:
# customise DataLoader
class LPDataset(Dataset):
    """
    A standard PyTorch definition of Dataset which defines the functions 
    __len__ and __getitem__.
    """
    def __init__(self, path, cv_idx, transform):
        """
        Store the filenames of the jpgs to use. 
        Specifies transforms to apply on images.

        Args:
            path: (string) directory containing the dataset
            cv_idx: cross validation indices (training / validation sets)
            transform: (torchvision.transforms) transformation to apply on image
        """
        self.filenames = [os.listdir(path)[i] for i in cv_idx]
        self.filenames = [os.path.join(path, f) for f in self.filenames 
                          if f.endswith('.jpg')]

        self.labels = [filename.split('/')[-1].split('_')[-1].split('.')[0] 
                       for filename in self.filenames]
        self.transform = transform

    def __len__(self):
        # return size of dataset
        return len(self.filenames)

    def __getitem__(self, idx):
        """
        Fetch index idx image and labels from dataset. 
        Perform transforms on image.

        Args:
            idx: (int) index in [0, 1, ..., size_of_dataset-1]

        Returns:
            image: (Tensor) transformed image
            label: corresponding label of image
        """
        image = Image.open(self.filenames[idx])  # PIL image
        image = self.transform(image)
        return image, self.labels[idx]


In [9]:
train_loader = DataLoader(LPDataset(PATH_TRAIN, train_idx, train_transformer), 
                          batch_size=BATCH_SIZE,  
                          shuffle=True)

In [10]:
val_set = LPDataset(PATH_TRAIN, val_idx, train_transformer)

### main.py

In [3]:
# manualSeed = random.randint(1, 10000)  # fix seed
# print("Random Seed: ", manualSeed)
# random.seed(manualSeed)
# np.random.seed(manualSeed)
# torch.manual_seed(manualSeed)
cudnn.benchmark = True

In [12]:
classes = string.ascii_uppercase+string.digits
nclass = len(classes) + 1
# number of channels 1=grayscale
nc = 1

In [13]:
converter = utils.strLabelConverter(classes)
criterion = nn.CTCLoss()

In [15]:
# # custom weights initialization called on crnn
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv') != -1:
#         m.weight.data.normal_(0.0, 0.02)
#     elif classname.find('BatchNorm') != -1:
#         m.weight.data.normal_(1.0, 0.02)
#         m.bias.data.fill_(0)

In [14]:
PRE_TRAINED_PATH = '../model_weights/CRNN30.pth'

In [15]:
# CRNN(imgH, nc, nclass, num_hidden(LSTM))
crnn = model.CRNN(IMGH, nc, nclass, 256)
crnn = torch.nn.DataParallel(crnn, range(1))

if torch.cuda.is_available():
    crnn = crnn.cuda()
    crnn.load_state_dict(torch.load(PRE_TRAINED_PATH))
else:
    crnn.load_state_dict(torch.load(PRE_TRAINED_PATH, map_location='cpu'))

In [16]:
image = torch.FloatTensor(BATCH_SIZE, 1, IMGH, IMGH)

if torch.cuda.is_available():
    image.cuda()
    
text = torch.IntTensor(BATCH_SIZE * 5)
length = torch.IntTensor(BATCH_SIZE)

In [17]:
# Learning Rate, Learning Rate Scheduler, Optimiser
LR = 1e-1
optimizer = optim.Adadelta(crnn.parameters(), lr=LR)
T_max = len(train_loader) * EPOCH
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max, eta_min=LR/10)

In [18]:
loss_avg = utils.averager()

In [19]:
def trainBatch(net, criterion, optimizer):
    data = train_iter.next()
    cpu_images, cpu_texts = data
    batch_size = cpu_images.size(0)
    utils.loadData(image, cpu_images)
    t, l = converter.encode(cpu_texts)
    utils.loadData(text, t)
    utils.loadData(length, l)
    
    preds = crnn(image)
    preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
    cost = criterion(preds, text, preds_size, length) / batch_size
    crnn.zero_grad()
    cost.backward()
    optimizer.step()
    lr_scheduler.step()
    return cost

In [20]:
def validation(net, dataset, criterion, max_iter=100):
    """
    To compute the validation loss from a given validation dataset
    
    net: neural network architecture
    dataset: validation set
    criterion: loss function
    max_iter: maximum number of mini_batches
    
    return: validation loss
    """
    
    for p in crnn.parameters():
        p.requires_grad = False
    
    # configure the mode: evaluation (model.train() & model.eval() behaves differently)
    net.eval()
    data_loader = DataLoader(
        dataset, shuffle=True, batch_size=BATCH_SIZE)
    val_iter = iter(data_loader)

    i = 0
    loss_avg = utils.averager()
    
    max_iter = min(max_iter, len(data_loader))
    for i in range(max_iter):
        data = val_iter.next()
        i += 1
        cpu_images, cpu_texts = data
        batch_size = cpu_images.size(0)
        utils.loadData(image, cpu_images)
        t, l = converter.encode(cpu_texts)
        utils.loadData(text, t)
        utils.loadData(length, l)

        preds = crnn(image)
        preds_size = Variable(torch.IntTensor([preds.size(0)] * batch_size))
        cost = criterion(preds, text, preds_size, length) / batch_size
        loss_avg.add(cost)

    return loss_avg.val()

In [23]:
# 25000 * 0.8 (# of data) // 64 (bs) ~= 310 (iterations) 
# NOTE: If no output => the training dataset is not fully loaded yet (i never reached 390)
display_iter = len(os.listdir(PATH_TRAIN)) * 0.8 // BATCH_SIZE

for epoch in trange(EPOCH):
    train_iter = iter(train_loader)
    i = 0

    while i < len(train_loader):
        for p in crnn.parameters():
            p.requires_grad = True
        crnn.train()
        
        cost = trainBatch(crnn, criterion, optimizer)
        loss_avg.add(cost)
        i += 1
        if i % display_iter == 0:
            # print training loss and validation loss
            print('[%d/%d][%d/%d] Train Loss: %f  Validation Loss: %f' %
                  (epoch, 30, i, len(train_loader), loss_avg.val(), validation(crnn, val_set, criterion)))
            loss_avg.reset()
            torch.save(crnn.state_dict(), 'experiments/netCRNN_{0}_{1}.pth'.format(epoch, i))   



  0%|          | 0/30 [00:00<?, ?it/s]

[0/30][100/110] Train Loss: 0.001483  Validation Loss: 0.001088




  3%|▎         | 1/30 [00:16<07:45, 16.07s/it]

[1/30][100/110] Train Loss: 0.001425  Validation Loss: 0.001105




  7%|▋         | 2/30 [00:32<07:29, 16.05s/it]

...



 93%|█████████▎| 28/30 [07:28<00:32, 16.02s/it]

[28/30][100/110] Train Loss: 0.000396  Validation Loss: 0.001098




 97%|█████████▋| 29/30 [07:44<00:16, 16.02s/it]

[29/30][100/110] Train Loss: 0.000392  Validation Loss: 0.000862




100%|██████████| 30/30 [08:00<00:00, 16.01s/it]